In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
!pip install pysmiles
from pysmiles import read_smiles
import networkx as nx
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ratelimit

  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5893 sha256=80676998074ec899387d67a9a04ab1202424e385876c9548548683bdbc37d328
  Stored in directory: /root/.cache/pip/wheels/27/5f/ba/e972a56dcbf5de9f2b7d2b2a710113970bd173c4dcd3d2c902
Successfully built ratelimit


In [ ]:
from ratelimit import limits

import requests
TIME_PERIOD = 1   # time period in seconds

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/summaryPubChemLarge.csv')

<ipython-input-5-4800a610c9b3>:1: DtypeWarning: Columns (2,32,33,35,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/summaryPubChemLarge.csv')


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def has_more_chars(string):
  count_numbers = 0
  count_characters = 0
  for character in string:
    if character.isdigit():
      count_numbers += 1
    else:
      count_characters += 1
  return count_characters > count_numbers

def simplify(arr):
  new_arr = arr.copy()

  for i in range(len(arr)):
    try:
      if 'doi' in str(arr[i]).lower():
        new_arr.remove(arr[i])
    except IndexError:
      pass

  for i in range(len(new_arr)):
    new_arr[i] = str(new_arr[i]).replace("<String>", "").replace("</String>", "")

  f_arr = []
  not_allowed = ['Link to all deposited patent identifiers', 'Patents are available for this chemical structure:', '(The corresponding statement to each P-code can be found at the GHS Classification page.)', 'Subscription Services', 'Legacy Depositors']
  forbidden = ['©', 'Inc.', 'The GHS information provided by', 'PubMed']
  for i in range(len(new_arr)):
    if (len(new_arr[i].split(' ')) >= 2 or ' is ' in new_arr[i]) and not new_arr[i].count(':') > 3 and not new_arr[i].count('+') > 3 and not any([f in new_arr[i] for f in forbidden]) and new_arr[i] not in not_allowed:
      f_arr.append(new_arr[i])
  if len(str.join(' ', f_arr).split(' ')) > 5:
    return list(set(f_arr))
  else:
    return 'None'

# @limits(calls=5, period=TIME_PERIOD)
def gather_data(cid):
  data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/xml")
  html = BeautifulSoup(data.content, "xml")
  return simplify(html.find_all("String"))

In [ ]:
gather_data(2244)

'None'

In [ ]:
data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{2244}/xml")
BeautifulSoup(data.content, "xml")

<?xml version="1.0" encoding="utf-8"?>
<Fault xmlns="http://pubchem.ncbi.nlm.nih.gov/pug_view" xmlns:xs="http://www.w3.org/2001/XMLSchema-instance" xs:schemaLocation="http://pubchem.ncbi.nlm.nih.gov/pug_view https://pubchem.ncbi.nlm.nih.gov/pug_view/pug_view.xsd">
<Code>PUGVIEW.ServerBusy</Code>
<Message>Too many requests or server too busy</Message>
</Fault>

In [ ]:
import time
import pickle
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

def scrape_pubchem(df):
  df['description'] = np.array([np.nan for i in range(len(df['cid']))])
  st = time.time()
  good = 0
  bad = 0
  for i in range(len(df['cid'])):
    if good >= 125125:
      break
    if (i+1) % 10 == 0:
      time_diff = (time.time()-st)/60
      pct = round((i+1)/len(df), 5)
      eta = (time_diff / pct) - time_diff
      clear_output()
      print(f'Finished {i+1} out of {len(df)}, {round(pct * 100, 2)}% done, Time gone by: {time_diff // 60} hours and {round(time_diff % 60, 2)} minutes, ETA: {eta // 60} hours and {round(eta % 60, 2)} minutes, Good: {good}, Bad:{bad}')

    if (i+1) % 5000 == 0:
      print('Making checkpoint')
      pickle.dump(df, open(f'drive/MyDrive/eda_summary_saves/description_checkpoint.v3.{i+1}.pkl', 'wb'))
    x = gather_data(df['cid'][i])
    if x != 'None':
      df['description'][i] = x
      good += 1
    else:
      bad += 1
  return df

In [ ]:
def scrape_pubchem_threaded(df):
  df['description'] = np.array([None for i in range(len(df['cid']))])
  good = 0
  bad = 0
  st = time.time()
  threads = 25
  for i in range(len(df['cid']) // threads):
    if i % 100 == 0:
      print('saving')
      pickle.dump(df, open(f'drive/MyDrive/eda_summary_saves/save{i*200}_threaded.pkl', 'wb'))
    time.sleep(0.1)
    with ThreadPoolExecutor(threads) as executor:
      # submit tasks and collect futures
      futures = [executor.submit(gather_data, df['cid'][i]) for i in range(threads)]
      # process task results as they are available
      for future in as_completed(futures):
          # retrieve the result
          result = future.result()
          if result != 'None':
            df['description'][i] = result
            good += 1
          else:
            bad += 1
      time_diff = (time.time()-st)/60
      pct = round((i+1)/(len(df)//threads), 5)
      eta = (time_diff / (pct)) - time_diff
      clear_output()
      print(f'Finished {i+1} threads out of {len(df)//threads}, {round(pct * 100, 2)}% done\nTime gone by: {time_diff // 60} hours and {round(time_diff % 60, 2)} minutes, ETA: {eta // 60} hours and {round(eta % 60, 2)} minutes, Good: {good}, Bad:{bad}')

  return df


In [ ]:
from time import sleep
from random import random
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [ ]:
x = scrape_pubchem_threaded(df)

In [ ]:
import concurrent
ids = [df['cid'][i] for i in range(1000)]
with ThreadPoolExecutor(1000) as executor:
   executor.map(gather_data, ids)

In [ ]:
# SuperFastPython.com
# example of getting results for tasks as they are completed


# start the thread pool
with ThreadPoolExecutor(10) as executor:
    # submit tasks and collect futures
    futures = [executor.submit(gather_data, df['cid'][i]) for i in range(10)]
    # process task results as they are available
    for future in as_completed(futures):
        # retrieve the result
        result = future.result()
        print(result)

In [ ]:
pickle.dump(x, open(f'drive/MyDrive/eda_summary_saves/description_checkpoint_large.pkl', 'wb')

In [ ]:
x = x['description'].dropna()

In [ ]:
x[23]